In [1]:
import json
from dataclasses import dataclass

import bigbench.api.results as bb

# Creating test set

Things to keep in mind:
- Instances are used for multiple systems. These multiple results should not be split across train and test.
- Instances are used for multiple n-shots. These multiple results should not be split across train and test. 

In [66]:
with open("../artifacts/logs/social_iqa/full-log_BIG-G_2m_T=0.json") as logs_raw:
    logs_json = json.load(logs_raw)
    logs: bb.ResultsFileData = bb.ResultsFileData.fromdict(logs_json, include_queries=True)
    
logs.model
logs.queries[0]

def print_sample(sample: bb.MultipleChoiceSample):
    print(sample.input)
    print(sample.correct)
    print(sample.target_values)

accuracy = sum(map(lambda s: s.correct, logs.queries[0].samples)) / len(logs.queries[0].samples) * 100
test_accuracy = sum(map(lambda s: s.correct, logs.queries[0].samples[900:])) / len(logs.queries[0].samples[900:]) * 100
print(len(logs.queries[3].samples), accuracy, test_accuracy)
for s in logs.queries[0].samples[:1]:
    print_sample(s)
    print("----------")

1024 32.91015625 29.838709677419356

Q: Taylor got married but decided to keep her identity and last name. How would you describe Taylor? 
  choice: Independent
  choice: Like a feminist
  choice: Shy and reticent
A: 
1.0
{'Independent': 1, 'Like a feminist': 0, 'Shy and reticent': 0}
----------


In [57]:
from datasets import Dataset, DatasetDict
import pandas as pd

zero_shot = logs.queries[0].samples
zero_shot = map(lambda i: {'text': i.input, 'label': int(i.correct)}, zero_shot)

ds = DatasetDict()
df = pd.DataFrame(zero_shot)
ds['train'] = Dataset.from_pandas(df.iloc[:900,:], split='train')
ds['test'] = Dataset.from_pandas(df.iloc[900:,:],split='test')

dataset = ds

In [58]:
# from datasets import load_dataset
yelp = load_dataset("yelp_review_full")
# yelp["train"][100]

dataset["train"][100]

Reusing dataset yelp_review_full (/home/wout/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

{'text': '\nQ: Cameron went out of their way to help a friend who was in need of help. How would Cameron feel after helping? \n  choice: Annoyed\n  choice: Frustrated\n  choice: Happy\nA: ',
 'label': 1}

In [59]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/wout/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/mai

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [60]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)
len(small_train_dataset), len(small_eval_dataset)

(900, 124)

In [67]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer",  evaluation_strategy="epoch", num_train_epochs=10)
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/wout/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/res

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.651736,0.701613
2,No log,0.625194,0.701613
3,No log,0.649247,0.709677
4,No log,0.815393,0.572581
5,No log,0.873230,0.661290
6,No log,1.431253,0.693548
7,No log,1.646531,0.709677
8,No log,1.792858,0.677419
9,0.310200,1.887069,0.669355
10,0.310200,1.985700,0.653226


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 124
  Batch size = 8
/home/wout/pp/llm-assessors/.env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 124
  Batch size = 8
/home/wout/pp/llm-assessors/.env/lib/python3.8/si

TrainOutput(global_step=570, training_loss=0.27276717523733773, metrics={'train_runtime': 171.7958, 'train_samples_per_second': 52.388, 'train_steps_per_second': 3.318, 'total_flos': 2367999498240000.0, 'train_loss': 0.27276717523733773, 'epoch': 10.0})